In [1]:
import scanpy
import numpy as np

In [2]:
import pandas as pd
import scanpy as sc
import pandas as pd
from scipy.sparse import csc_matrix
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

from sklearn.metrics import matthews_corrcoef

# Read the AnnData object
data = sc.read_h5ad("/Users/sidvash/github/Hackathon-Summer-2023/test_data/sce.h5ad")

# Access the data matrix from the AnnData object
data_matrix = data.X

dense_array = data_matrix.toarray()

# Get the list of gene names
gene_names = data.var_names

# Convert the data matrix to a Pandas DataFrame
features_df = pd.DataFrame(dense_array, columns=gene_names, index=data.obs_names)

In [3]:
features_df.head(2)

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A4GALT,AA415398,AA465934,AA467197,...,hsa-mir-3181,hsa-mir-335,hsa-mir-4259,hsa-mir-4537,hsa-mir-4538,hsa-mir-490,hsa-mir-5195,hsa-mir-6080,hsa-mir-8072,snoU2-30
AGAGCGAAGATCTGCT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
ACATCAGTCTGACCTC,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
features_df.columns

Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A4GALT',
       'AA415398', 'AA465934', 'AA467197',
       ...
       'hsa-mir-3181', 'hsa-mir-335', 'hsa-mir-4259', 'hsa-mir-4537',
       'hsa-mir-4538', 'hsa-mir-490', 'hsa-mir-5195', 'hsa-mir-6080',
       'hsa-mir-8072', 'snoU2-30'],
      dtype='object', length=25269)

In [5]:
def check_if_col_remove(column_name,
                        dataset,
                        threshold=0.99):
    """
    Check if a column should be removed from the dataset
    based on the percentage of 0 values.

    Parameters
    ----------
    column_name : str
        Name of the column to check.
    dataset : pandas.DataFrame
        Dataset to check.
    threshold : float
        Threshold to use to check if the column should be removed.
        0.9 means that if 90% of the values are 0, the column will be removed.

    Returns
    -------
    bool
        True if the column should be removed, False otherwise.
    """
    # Get the number of rows in the dataset
    num_rows = dataset.shape[0]

    # Get the number of 0 values in the column
    num_zeros = dataset[column_name].value_counts()[0]

    # Check if the number of 0 values is greater than the threshold
    if num_zeros / num_rows > threshold:
        # print(f"ratio: {num_zeros}/{num_rows}: {num_zeros / num_rows}")
        return True
    else:
        return False


In [6]:
threshold_values = [0.99, 0.95, 0.90, 0.85, 0.80, 0.75, 0.70]


threshold2removedColumns = {}

for threshold in threshold_values:
    columns_to_remove = set()
    for column in tqdm(list(features_df.columns)):
        if check_if_col_remove(column,
                                features_df,
                                threshold=threshold):
            columns_to_remove.add(column)

    threshold2removedColumns[threshold] = columns_to_remove

100%|██████████| 25269/25269 [00:05<00:00, 4980.30it/s]


In [7]:
for threshold in threshold_values:
    print(f"At threshold {threshold}, we remove {len(threshold2removedColumns[threshold])} columns")

At threshold 0.99, we remove 16284 columns
At threshold 0.95, we remove 22888 columns
At threshold 0.9, we remove 24405 columns
At threshold 0.85, we remove 24788 columns
At threshold 0.8, we remove 24964 columns
At threshold 0.75, we remove 25040 columns
At threshold 0.7, we remove 25094 columns


## Train Data

## Sample indexes from train

In [8]:
supervised_df = pd.read_csv("../train/training.csv", header=None)
sampleid2label = dict(zip(supervised_df[0].values, supervised_df[1].values))

# rename columns 0: sample_id, 1: label
supervised_df.columns = ['sample_id', 'label']

supervised_df


,sample_id,label
0,GCTCTGTCAATGGATA,1
1,GATGAGGGTACGAAAT,0
2,AGGCCACGTACAGCAG,0
3,ACGCCGAGTCACACGC,0
4,TCTTCGGAGGCTAGCA,0
...,...,...
995,ACTGCTCCACTCGACG,0
996,TGGTTAGGTAAACGCG,0
997,TTCTCAATCAGTACGT,1
998,TGAGCCGGTCTCTTAT,0


In [9]:
all_supervised_samples = list(supervised_df['sample_id'].values)


## Great value seeds: (upto mcc 58)
# SEED = 42
# dev_ratio = 0.05
# test_ratio = 0.05

SEED = 42
dev_ratio = 0.1
test_ratio = 0.1

threshold_values = [0.99, 0.95, 0.90, 0.85, 0.80, 0.75, 0.70]

dev_samples = random.Random(SEED).sample(all_supervised_samples, int(len(all_supervised_samples) * dev_ratio))
test_samples = random.Random(SEED).sample(list(set(all_supervised_samples) - set(dev_samples)), int(len(all_supervised_samples) * test_ratio))
train_samples = set(all_supervised_samples) - set(dev_samples) - set(test_samples)

print(len(dev_samples), len(test_samples), len(train_samples))

100 100 800


In [10]:
## select only rows where the index matches the train_sample_set
train_df = features_df[features_df.index.isin(train_samples)]
dev_df = features_df[features_df.index.isin(dev_samples)]
test_df = features_df[features_df.index.isin(test_samples)]

y_train = np.array([sampleid2label[sample] for sample in train_df.index])
y_dev = np.array([sampleid2label[sample] for sample in dev_df.index])
y_test = np.array([sampleid2label[sample] for sample in test_df.index])


In [11]:
sum(y_test)/len(y_test)

0.14

In [12]:
sum(y_dev)/len(y_dev)

0.16

In [13]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

## FINDING BEST MCC

In [14]:
# def mcc(tp, tn, fp, fn):
#     sup = tp * tn - fp * fn
#     inf = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
#     if inf==0:
#         return 0
#     else:
#         return sup / np.sqrt(inf)


# def eval_mcc(y_true, y_prob, show=False):
#     idx = np.argsort(y_prob)
#     y_true_sort = y_true[idx]
#     n = y_true.shape[0]
#     nump = 1.0 * np.sum(y_true) # number of positive
#     numn = n - nump # number of negative
#     tp = nump
#     tn = 0.0
#     fp = numn
#     fn = 0.0
#     best_mcc = 0.0
#     best_id = -1
#     prev_proba = -1
#     best_proba = -1
#     mccs = np.zeros(n)
#     for i in range(n):
#         # all items with idx < i are predicted negative while others are predicted positive
#         # only evaluate mcc when probability changes
#         proba = y_prob[idx[i]]
#         if proba != prev_proba:
#             prev_proba = proba
#             new_mcc = mcc(tp, tn, fp, fn)
#             if new_mcc >= best_mcc:
#                 best_mcc = new_mcc
#                 best_id = i
#                 best_proba = proba
#         mccs[i] = new_mcc
#         if y_true_sort[i] == 1:
#             tp -= 1.0
#             fn += 1.0
#         else:
#             fp -= 1.0
#             tn += 1.0

#     return best_mcc
#     # if show:
#     #     y_pred = (y_prob >= best_proba).astype(int)
#     #     score = matthews_corrcoef(y_true, y_pred)
#     #     print(score, best_mcc)
#     #     plt.plot(mccs)
#     #     return best_proba, best_mcc, y_pred
#     # else:
#     #     return best_mcc



# def mcc_eval(y_prob, dtrain):
#     y_true = dtrain.get_label()
#     best_mcc = eval_mcc(y_true, y_prob)
#     return 'MCC', best_mcc

In [15]:
# # convert all float values to int
# df_train_at_threshold = df_train.drop(columns=list(threshold2removedColumns[threshold]))
# print(f"Feature sizes: {df_train_at_threshold.shape[1]}")

# df_train_at_threshold = df_train_at_threshold.astype(int)
# df_train_at_threshold.values

In [16]:
# random state 123 gives 0.435 mcc
threshold2Model = {}
for threshold in threshold_values:
    print("################################################")
    print("Threshold: ", threshold)
    # drop columns that are not needed for this threshold value
    train_df_at_threshold = train_df.drop(columns=list(threshold2removedColumns[threshold]))
    dev_df_at_threshold = dev_df.drop(columns=list(threshold2removedColumns[threshold]))
    test_df_at_threshold = test_df.drop(columns=list(threshold2removedColumns[threshold]))

    print(f"Feature sizes (train): {train_df_at_threshold.shape[1]}")
    print(f"Feature sizes (dev): {dev_df_at_threshold.shape[1]}")
    print(f"Feature sizes (test): {test_df_at_threshold.shape[1]}")

    X_train = train_df_at_threshold.values
    X_val = dev_df_at_threshold.values
    X_test = test_df_at_threshold.values


    # X_train, X_test, y_train, y_test = train_test_split(df_train_at_threshold.values, y, test_size=0.2, random_state=123)
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=123) # 0.25 x 0.8 = 0.2


    # Computing class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

    # Creating the XGBoost DMatrix for training and validation
    dtrain = xgb.DMatrix(X_train, label=y_train, weight=class_weights[y_train])
    dval = xgb.DMatrix(X_val, label=y_dev)

    # Defining the XGBoost parameters
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'eta': 0.1,
        'max_depth': 3,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'scale_pos_weight': class_weights[0] / class_weights[1]  # Adjusting for class imbalance
    }

    # Training the XGBoost model
    threshold2Model[threshold] = xgb.train(params, dtrain, num_boost_round=100, early_stopping_rounds=10, evals=[(dval, 'validation')])


    # Predicting on the test set
    dtest = xgb.DMatrix(X_test)


    y_pred_val = threshold2Model[threshold].predict(dval)
    y_pred_val = (y_pred_val > 0.3).astype(int)

    y_pred = threshold2Model[threshold].predict(dtest)
    # y_pred = model.predict(dtest)
    y_pred = (y_pred > 0.3).astype(int)

    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print(f"Accuracy: {accuracy}")
    print("Classification Report:")
    print(report)


    mcc = matthews_corrcoef(y_test, y_pred)
    mcc_val = matthews_corrcoef(y_dev, y_pred_val)

    print(f"The mcc value on val set is: {mcc_val}")
    print(f"The mcc value on test set is: {mcc}")
    

################################################
Threshold:  0.99
Feature sizes (train): 8985
Feature sizes (dev): 8985
Feature sizes (test): 8985
[0]	validation-logloss:0.63825
[1]	validation-logloss:0.59475
[2]	validation-logloss:0.55551
[3]	validation-logloss:0.52769
[4]	validation-logloss:0.50470
[5]	validation-logloss:0.48082
[6]	validation-logloss:0.46358
[7]	validation-logloss:0.44886
[8]	validation-logloss:0.43784
[9]	validation-logloss:0.42809
[10]	validation-logloss:0.41564
[11]	validation-logloss:0.40620
[12]	validation-logloss:0.39910
[13]	validation-logloss:0.39296
[14]	validation-logloss:0.38589
[15]	validation-logloss:0.37946
[16]	validation-logloss:0.37380
[17]	validation-logloss:0.36872
[18]	validation-logloss:0.36864
[19]	validation-logloss:0.36613
[20]	validation-logloss:0.36360
[21]	validation-logloss:0.36313
[22]	validation-logloss:0.35903
[23]	validation-logloss:0.35834
[24]	validation-logloss:0.35741
[25]	validation-logloss:0.35595
[26]	validation-logloss:0.35499

## Ensemble of Models

In [17]:
# threshold_values = [0.90, 0.85]

In [18]:

## create a np.array of zeros with the same shape as test set
predictions = np.zeros(len(y_test),)
predictions_val = np.zeros(len(y_dev),)
predictions_train = np.zeros(len(y_train),)

for threshold in threshold_values:

    print("Threshold: ", threshold)
    # drop columns that are not needed for this threshold value
    train_df_at_threshold = train_df.drop(columns=list(threshold2removedColumns[threshold]))
    dev_df_at_threshold = dev_df.drop(columns=list(threshold2removedColumns[threshold]))
    test_df_at_threshold = test_df.drop(columns=list(threshold2removedColumns[threshold]))

    print(f"Feature sizes (train): {train_df_at_threshold.shape[1]}")
    print(f"Feature sizes (dev): {dev_df_at_threshold.shape[1]}")
    print(f"Feature sizes (test): {test_df_at_threshold.shape[1]}")

    X_train = train_df_at_threshold.values
    X_val = dev_df_at_threshold.values
    X_test = test_df_at_threshold.values

    # Creating the XGBoost DMatrix for training and validation
    dtrain = xgb.DMatrix(X_train, label=y_train, weight=class_weights[y_train])
    dval = xgb.DMatrix(X_val, label=y_dev)

    # Predicting on the test set
    dtest = xgb.DMatrix(X_test)
    predictions += threshold2Model[threshold].predict(dtest)
    predictions_val += threshold2Model[threshold].predict(dval)
    predictions_train += threshold2Model[threshold].predict(dtrain)

        
## Average the predictions
predictions /= len(threshold_values)
predictions_val /= len(threshold_values)
predictions_train /= len(threshold_values)

y_pred = (predictions > 0.5).astype(int)
y_pred_val = (predictions_val > 0.5).astype(int)
y_pred_train = (predictions_train > 0.5).astype(int)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(y_test, y_pred)
mcc_val = matthews_corrcoef(y_dev, y_pred_val)
mcc_train = matthews_corrcoef(y_train, y_pred_train)

print(f"The mcc value on test set is: {mcc}")
print(f"The mcc value on dev set is: {mcc_val}")
print(f"The mcc value on train set is: {mcc_train}")

Threshold:  0.99
Feature sizes (train): 8985
Feature sizes (dev): 8985
Feature sizes (test): 8985
Threshold:  0.95
Feature sizes (train): 2381
Feature sizes (dev): 2381
Feature sizes (test): 2381
Threshold:  0.9
Feature sizes (train): 864
Feature sizes (dev): 864
Feature sizes (test): 864
Threshold:  0.85
Feature sizes (train): 481
Feature sizes (dev): 481
Feature sizes (test): 481
Threshold:  0.8
Feature sizes (train): 305
Feature sizes (dev): 305
Feature sizes (test): 305
Threshold:  0.75
Feature sizes (train): 229
Feature sizes (dev): 229
Feature sizes (test): 229
Threshold:  0.7
Feature sizes (train): 175
Feature sizes (dev): 175
Feature sizes (test): 175
Accuracy: 0.89
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.99      0.94        86
           1       0.80      0.29      0.42        14

    accuracy                           0.89       100
   macro avg       0.85      0.64      0.68       100
weighted avg       0.8

## Predict on test data

In [19]:
predictions = pd.read_csv('../prediction/prediction.csv', header=None)
prediction_sample_ids = predictions[0].values

In [20]:
predictions_df = features_df[features_df.index.isin(prediction_sample_ids)].reset_index(drop=True)

In [21]:
predictions_df.shape[0]

20954

In [22]:
predictions_df.shape

(20954, 25269)

In [23]:
sum(predictions[1])

2716

In [24]:
## create a np.array of zeros with the same shape as test set
predictions_final = np.zeros(predictions_df.shape[0],)

for threshold in threshold_values:
    print("Threshold: ", threshold)
    # drop columns that are not needed for this threshold value
    predictions_df_at_threshold = predictions_df.drop(columns=list(threshold2removedColumns[threshold]))
    
    print(f"Feature sizes (train): {predictions_df_at_threshold.shape[1]}")
    
    X = predictions_df_at_threshold.values
    
    # Predicting on the test set
    dtest = xgb.DMatrix(X)
    predictions_final += threshold2Model[threshold].predict(dtest)
    
## Average the predictions
predictions_final /= len(threshold_values)

y_pred = (predictions_final > 0.3).astype(int)


Threshold:  0.99
Feature sizes (train): 8985
Threshold:  0.95
Feature sizes (train): 2381
Threshold:  0.9
Feature sizes (train): 864
Threshold:  0.85
Feature sizes (train): 481
Threshold:  0.8
Feature sizes (train): 305
Threshold:  0.75
Feature sizes (train): 229
Threshold:  0.7
Feature sizes (train): 175


In [26]:
sum(y_pred)

2425

In [27]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [29]:
predictions_df

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A4GALT,AA415398,AA465934,AA467197,...,hsa-mir-3181,hsa-mir-335,hsa-mir-4259,hsa-mir-4537,hsa-mir-4538,hsa-mir-490,hsa-mir-5195,hsa-mir-6080,hsa-mir-8072,snoU2-30
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
with open("final_predictions.csv", "w") as f:
    for id, pred in zip(prediction_sample_ids, y_pred):
        f.write(f"{id},{pred}\n")
